In [44]:
!mkdir GloVe
!curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip GloVe/glove.840B.300d.zip -d GloVe/
!mkdir fastText
!curl -Lo fastText/crawl-300d-2M.vec.zip https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip fastText/crawl-300d-2M.vec.zip -d fastText/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   353    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2075M  100 2075M    0     0  5187k      0  0:06:49  0:06:49 --:--:-- 5117k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1453M  100 1453M    0     0  45.0M      0  0:00:32  0:00:32 --:--:-- 51.5M
Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [45]:
!mkdir encoder
!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  53.4M      0  0:00:02  0:00:02 --:--:-- 53.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  38.8M      0  0:00:03  0:00:03 --:--:-- 38.8M


In [46]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
from models import InferSent
V = 2
model_version = 1
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [48]:
use_cuda = False
infersent = infersent.cuda() if use_cuda else infersent


In [38]:
# W2V_PATH = 'fastText/crawl-300d-2M.vec'
# infersent.set_w2v_path(W2V_PATH)
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)


In [59]:
with open('STS2016.input.plagiarism.txt') as f:
    lines = str(f.readlines())
s1=str(lines).split('\\n')
line1=[]
line2=[]
for i in range(0,len(s1)-1):
  line1.append(s1[i].split('\\t')[0])
  line2.append(s1[i].split('\\t')[1])
import re
string1=[]
for i in line1:
  string1.append(re.sub(r"[^a-zA-Z0-9]+",' ',i))
string2=[]
for i in line2:
  string2.append(re.sub(r"[^a-zA-Z0-9]+",' ',i))
strin=string1 +string2


In [60]:
infersent.set_w2v_path(W2V_PATH)
infersent.build_vocab(strin, tokenize=True)

import numpy as np
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

# infersent.build_vocab(string1, tokenize=True)
# infersent.build_vocab(string2, tokenize=True)

a = infersent.encode(string1, tokenize=True)
b = infersent.encode(string2, tokenize=True)
x=0
similar=[]
for i in range(0, len(string1)-1):
  # query = string1[i]
  # query2 = string2[i]
  # query_vec = infersent.encode(string1,tokenize=True)
  sim = cosine(a[i],b[i])
  print("Sentence = ", string1[i], string2[i], "; similarity = ", sim)
  similar.append(sim)
  x=x+1


Found 925(/934) words with w2v vectors
Vocab size : 925
Sentence =   The vector space model has some limitations 1  The vector space model are the documents which are represented as bags of words  ; similarity =  0.9214582
Sentence =   Secondly to define the value of the optimal solution recursively  Define value of optimal solution recursively  ; similarity =  0.9397198
Sentence =   Vector space representation results in the loss of the order which the terms are in the document  If a term occurs in the document the value will be non zero in the vector  ; similarity =  0.8884237
Sentence =   P B is the prior or marginal probability of B and acts as a normalizing constant  P A or the probability that the student is a girl regardless of any other information  ; similarity =  0.8925175
Sentence =   It is prior in the sense that it does not take into account any information about B  It is previous in the sense that it does not take into account any information about B  ; similarity =  0.99

In [61]:
print(similar)

[0.9214582, 0.9397198, 0.8884237, 0.8925175, 0.9941886, 0.8845271, 0.9247198, 0.9154678, 0.98602766, 0.9174961, 0.8560316, 0.8811093, 0.9435322, 0.93914425, 0.9028002, 0.8526482, 0.8887413, 0.88960725, 0.9334207, 0.856763, 0.9882701, 0.9425054, 0.8877229, 0.9882701, 0.79728055, 0.9041914, 0.9827551, 0.99391305, 0.8726544, 0.9489912, 0.8695483, 0.90849286, 0.9482927, 0.84145856, 0.87464684, 0.8482825, 0.9581391, 0.9113696, 0.878877, 0.8355092, 0.97354186, 0.8628417, 0.8961195, 0.91219336, 0.84725237, 0.89176893, 0.8922736, 0.96675277, 0.9042004, 0.86363906, 0.8696925, 0.86605394, 0.90177643, 0.9047573, 0.9818371, 0.9556485, 0.8680735, 0.91203487, 0.910522, 0.91908467, 0.8331318, 0.85348344, 0.9126765, 0.89619035, 0.93835884, 0.826916, 0.94609815, 0.93130153, 0.8853333, 0.8285496, 0.8845716, 0.90425766, 0.9004721, 0.87262017, 0.8557438, 0.89322746, 0.8575879, 0.8501162, 0.86132926, 0.9316801, 0.92899394, 0.91991204, 0.9624978, 0.9177559, 0.9413996, 0.84904265, 0.9581391, 0.9042783, 0.875